In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError:
    strategy = tf.distribute.get_strategy() # for CPU and single GPU
    print('Number of replicas:', strategy.num_replicas_in_sync)

# Import Libraries

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import nltk
import nltk.corpus
import re
import string
from nltk.corpus import stopwords
import gensim

# Load Data

In [ ]:
train_data=pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
test_data=pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')
submission=pd.read_csv('/kaggle/input/nlp-getting-started/sample_submission.csv')

# Data preprocessing

In [ ]:
train_data.head()

In [ ]:
train_data=train_data.drop(["keyword","location"],axis=1)
test_data=test_data.drop(["keyword","location"],axis=1)

train_data.head(2)

In [ ]:
type(train_data)

In [ ]:
total_data= pd.concat([train_data,test_data],axis=0,ignore_index=True)

In [ ]:
total_data

In [ ]:
# handel urls
def remove_url(text):
    url=re.compile(r'https?://S+|www\.\S+')
    return url.sub(r'',text)
def remove_html(text):
    html = re.compile(r'<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    return re.sub(html, '', text)

In [ ]:
total_data['text']=total_data['text'].apply(lambda x:remove_url(x))
total_data['text']=total_data['text'].apply(lambda x:remove_html(x))

In [ ]:
# convert to lower alphabates
for i in range(len(total_data)):
    total_data['text'][i].lower()

In [ ]:
stop_words=set(stopwords.words("english"))
print(stop_words)

In [ ]:
#tokenize text
total_data['tokenized_text']=total_data['text'].apply(gensim.utils.simple_preprocess)

In [ ]:
total_data.head()

In [ ]:
# remove stopwords
def remove_stopwords(s):
    cleantext=[]
    for i in range(len(s)): 
        if(s[i] not in stop_words):
                cleantext.append(s[i])
    return cleantext


In [ ]:
total_data['clean_tok_text']=total_data['tokenized_text'].apply(lambda x:remove_stopwords(x))
total_data.head()

In [ ]:
total_data.shape

In [ ]:
# join clean tokenized text
def join_fun(text):
    string=" "
    ans=string.join(text)
    return ans
total_data['final_text']=total_data['clean_tok_text'].apply(lambda x:join_fun(x))
total_data.head()

# Predictions using Count Vectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()

vec=cv.fit_transform(total_data["final_text"])

In [ ]:
print(total_data.shape,vec.shape)

In [ ]:
X=vec[:len(train_data)]
y=train_data.target
X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.90)

In [ ]:
model=RandomForestClassifier(n_estimators=65)
model.fit(X_train,y_train)
pred=model.predict(X_test)
accuracy_score(pred,y_test)

# Using TfIdf Vectorizer
TF-IDF is better than Count Vectorizers because it not only focuses on the frequency of words present in the corpus but also provides the importance of the words. We can then remove the words that are less important for analysis, hence making the model building less complex by reducing the input dimensions.


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfvec=TfidfVectorizer()

vec1=tfidfvec.fit_transform(total_data["final_text"])

In [ ]:
X=vec1[:len(train_data)]
y=train_data.target
X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.90)

In [ ]:
model=RandomForestClassifier(n_estimators=65)
model.fit(X_train,y_train)
pred=model.predict(X_test)
accuracy_score(pred,y_test)

In [ ]:
test_vac=vec[len(train_data):]
test_vac

# BertTokenizier


In [ ]:
from transformers import BertTokenizer, TFBertModel, TFAutoModel,AutoTokenizer
model_name ='joeddav/xlm-roberta-large-xnli'
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
def encode_sentence(s):
    tokens=[]
    tokens.append('[CLS]')
    tokens+=list(tokenizer.tokenize(s))
#     print(tokens)
    return tokenizer.convert_tokens_to_ids(tokens)

In [ ]:
main_data=total_data['final_text']
main_data=pd.DataFrame(main_data)

In [ ]:
main_data.head()

In [ ]:
encode_sentence(main_data['final_text'][0])

In [ ]:
tokenized=[]
for i in range(len(main_data)):
    tokenized.append(encode_sentence(main_data['final_text'][i]))
main_data['tokenized']=tokenized

In [ ]:
main_data['target']=total_data['target']
main_data

# Attention Mask and Token Type ID:

In [ ]:
# see the documentations

mask=[]
for i in range(len(main_data)):
    padded_seq=tokenizer(main_data['final_text'][i],padding=True,add_special_tokens =True)
    mask.append(padded_seq)

main_data['masked']=mask
main_data.head()

# Train Model

In [ ]:
max_len=32
def build_model():
    bert_encoder = TFAutoModel.from_pretrained('joeddav/xlm-roberta-large-xnli')
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    embedding = bert_encoder([input_word_ids, input_mask])[0]
    output = tf.keras.layers.Dense(3, activation='softmax')(embedding[:,0,:])
    
    model = tf.keras.Model(inputs=[input_word_ids, input_mask], outputs=output)
    model.compile(tf.keras.optimizers.Adam(lr=1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
def input_convert(data):
    inputs={
        'input_word_ids':[],
        'input_mask':[]
    }
    for each in data:
        inputs['input_word_ids'].append(each['input_ids'])
        inputs['input_mask'].append(each['attention_mask'])
        
    
    inputs['input_word_ids']= tf.ragged.constant( inputs['input_word_ids']).to_tensor()
    inputs['input_mask']= tf.ragged.constant( inputs['input_mask']).to_tensor()
    return inputs

In [ ]:

data_for_predictions=main_data[len(train_data):]
data_for_fitting=main_data[:len(train_data)]
data_for_fitting
    

In [ ]:
train_input=input_convert(data_for_fitting['masked'].values)
for key in train_input.keys():
    train_input[key] = train_input[key][:,:max_len]

In [ ]:
train_input

In [ ]:
early_stop=tf.keras.callbacks.EarlyStopping(patience=3,restore_best_weights=True)
with strategy.scope():
    model=build_model()
    model.summary()
    model.fit(train_input,main_data['target'].values,epochs=5,verbose=1,batch_size=128,validation_split=0.1,callbacks=[early_stop])

# predictions

In [ ]:
test_input=input_convert(data_for_predictions['masked'].values)
for key in train_input.keys():
    test_input[key] = test_input[key][:,:max_len]

predictions=[np.argmax(i) for i in model.predict(test_input)]

In [ ]:
submission['target']=predictions
submission.to_csv("submission.csv",index=False)

In [ ]:
submission.head(10)